In [1]:
import os
import math

import time
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [2]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [3]:
def featureFactory(dataFrame):
    print('Feature Augmentation')
    minute = dataFrame.time%60
    dataFrame['hour'] = dataFrame['time'].div(60).map(int)
#     dataFrame.drop(['time'], axis=1, inplace=True)
    dataFrame['weekday'] = dataFrame['hour'].div(24).map(int)
    dataFrame['month'] = dataFrame['weekday'].div(30).map(int)
    dataFrame['day']=dataFrame['hour'].div(24).map(int)
    
    dataFrame['year'] = (dataFrame['weekday'].div(365).map(int)+1)*10.0
    dataFrame['hour'] = ((dataFrame.loc[:,'hour'].values%(24)+1)+minute.div(60.0))*(4.0)


    dataFrame['weekday'] = (dataFrame['weekday']%7+1)*3.1
    dataFrame['month'] = (dataFrame['month']%12+1)*2.1
    dataFrame['dayOfMonth']=(dataFrame['day']%30+1)
    dataFrame['accuracy'] = np.log10(dataFrame['accuracy'])*10.0
    dataFrame['originalIndex'] = np.arange(len(dataFrame))

    print ('processing is done')
f=['x','y', 'accuracy','hour', 'weekday', 'month', 'year']

In [4]:
def amplifier1(dataFrame):    # add data for periodic time that hit the boundary
    add_data = dataFrame[dataFrame.hour<6]
    add_data.hour = add_data.hour+96
    return add_data
    
def amplifier2(dataFrame):
    add_data = dataFrame[dataFrame.hour>98]
    add_data.hour = add_data.hour-96
    return add_data

In [5]:
featureFactory(train)
featureFactory(test)

# ap1=amplifier1(train)
# ap2=amplifier2(train)
# train=train.append(ap1).append(ap2)

train=train.append(amplifier1(train))
train=train.append(amplifier2(train))


# pd.options.mode.chained_assignment = None

Feature Augmentation
processing is done


/home/zhenjie/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Feature Augmentation
processing is done


In [ ]:
# test=pd.read_csv('../input/test.csv')
# featureFactory(test)

In [ ]:
# g=train.groupby('place_id')
# places=pd.DataFrame()
# places['xMean']=g['x'].mean()
# places['xStd']=g['x'].std()

# places['yMean']=g['y'].mean()
# places['yStd']=g['y'].std()

# sTime=time.time()

# train=train[(abs(train['y']-places.loc[train['place_id'], 'yMean'].values)<5*places.loc[train['place_id'], 'yStd'].values)\
#          & (abs(train['x']-places.loc[train['place_id'], 'xMean'].values)<5*places.loc[train['place_id'], 'xStd'].values)\
           
#            ]

# print time.time()-sTime

In [6]:
nXRegions=20
nYRegions=40
xMin=0
xMax=10.1
yMin=0
yMax=10.1
nXRegions=nXRegions*int(xMax-xMin)/10
nYRegions=nYRegions*int(yMax-yMin)/10
# train=train[(train['x']<xMax) & (train['x']>=xMin) & (train['y']<yMax) & (train['y']>=yMin)]
# valid=valid[(valid['x']<xMax) & (valid['x']>=xMin) & (valid['y']<yMax) & (valid['y']>=yMin)]

print xMin, xMax, yMin, yMax, nXRegions, nYRegions

0 10.1 0 10.1 20 40


In [ ]:
# def regionalForestPred(pred, classes, regionalPoints, data):

    
#     prob=[sorted(zip(p, classes))[-3:][::-1] for p in pred]
    
    
#     regionalPred=[]
#     for i in xrange(len(data)):
#         sampleY=data['y'].iloc[i]
#         sampleX=data['x'].iloc[i]
#         sampleDay=data['day'].iloc[i]

#         filteredPlaces=filter(lambda x: \
#                               sampleY<=yMaxMap[x[1]]+yAllowance and sampleY>=yMinMap[x[1]]-yAllowance and \
#                               sampleX<=xMaxMap[x[1]] and sampleX>=xMinMap[x[1]] and \
#                               sampleDay<=dayMaxMap[x[1]] and sampleDay>=dayMinMap[x[1]], prob[i])
#         if filteredPlaces:
#             filteredPlaces=map(lambda x: [x[0]*regionalPoints, x[1]], filteredPlaces)
#         regionalPred.append(filteredPlaces)
#     return regionalPred

In [7]:
def train_test(modelSerialization=False, resultSerialization=False, model='rf', predictSet=test, th=0, xMargin=0.05, yMargin=0.025):
    
    
    if modelSerialization or resultSerialization:
        base='./'
        folderName=model+'-'+time.strftime('%c')
        folderPath=base+folderName+'/'
        if not os.path.exists(folderPath):
            os.makedirs(folderPath)
            
    
#     f=['x','y','xTimeY','xDivY', 'accuracy', 'day', 'weekday', 'hourInDay','hourInDayShifted2', 'month', 'year']
#     f=['x','y', 'accuracy', 'day','weekday', 'hourInDay','hourInDayShifted2', 'month', 'year']
    f=['x','y', 'accuracy','hour', 'weekday','month', 'year']

        
    count=0
    accuracies=[]
    FBscores=[]
    
    startTime=time.time()
    trainingTimes=[]
    predictionTimes=[]
    
    testNum=0

    roundNum=5

    for xNum, yNum in ((xn, yn) for xn in xrange(nXRegions) for yn in xrange(nYRegions)):

        xStep=round(1.0*(xMax-xMin)/nXRegions,roundNum)
        xStart=round(xMin+xNum*xStep,roundNum)
        xEnd=round(xStart+xStep,roundNum)

        yStep=round(1.0*(yMax-yMin)/nYRegions,roundNum)
        yStart=round(yMin+yNum*yStep,roundNum)
        yEnd=round(yStart+yStep,roundNum)
        
#         regionalTrain=train[(train['x']<xEnd) & (train['x']>=xStart) & (train['y']<yEnd) & (train['y']>=yStart)]

        regionalTrain=train[(train['x']<xEnd+xMargin) & (train['x']>=xStart-xMargin) & (train['y']<yEnd+yMargin) & (train['y']>=yStart-yMargin)]
#         print xStart, xEnd, yStart, yEnd, len(regionalTrain)

        
            
#         regionalTrain['sampleWeight']=


        placesCounts=regionalTrain['place_id'].value_counts()
        regionalTrain=regionalTrain[(placesCounts[regionalTrain['place_id'].values]>=th).values]
#        this is no good, neight :tail or -tail:
#         tail=len(regionalTrain)/20
#         latest=regionalTrain.sort_values(by=['time'])[-tail:]
#         regionalTrain=regionalTrain.append(latest)
        
        
#         addition=regionalTrain[regionalTrain['hourInDay']<=4]
#         addition['hourInDay']=addition['hourInDay']+24
#         regionalTrain=regionalTrain.append(addition)
        
#         addition=regionalTrain[regionalTrain['hourInDay']>=20]
#         addition['hourInDay']=addition['hourInDay']-24
#         regionalTrain=regionalTrain.append(addition)


        
        if len(regionalTrain):
#             regionalPredictSet=predictSet[(predictSet['x']<xEnd-margin) & (predictSet['x']>=xStart+margin) & (predictSet['y']<yEnd-margin) & (predictSet['y']>=yStart+margin)]
            regionalPredictSet=predictSet[(predictSet['x']<xEnd) & (predictSet['x']>=xStart) & (predictSet['y']<yEnd) & (predictSet['y']>=yStart)]

            testNum+=len(regionalPredictSet)

            if model=='knn':
                regionalTrain.loc[:,'x'] *= 490.0
                regionalTrain.loc[:,'y'] *= 980.0
                regionalPredictSet.loc[:,'x'] *= 490.0
                regionalPredictSet.loc[:,'y'] *= 980.0

#                 fWeights={'x': 500, 'y': 1000, 'accuracy':0, 'day': 0,'dayInMonth': 0, 'weekday': 3,\
#                           'hourInDay': 4, 'hourInDayShifted2': 0, 'month': 2, 'year': 10}
#                 for ft in f:
#                     regionalTrain[ft]=regionalTrain[ft].values*fWeights[ft]
#                     regionalPredictSet[ft]=regionalPredictSet[ft].values*fWeights[ft]
                
            else:
                regionalTrain.loc[:,'x'] *= 490.0
                regionalTrain.loc[:,'y'] *= 980.0
                regionalPredictSet.loc[:,'x'] *= 490.0
                regionalPredictSet.loc[:,'y'] *= 980.0
                
#                 fMean=regionalTrain[f].mean()
#                 fStd=regionalTrain[f].std()

#                 for ft in f:       
#                     regionalTrain[ft]=(regionalTrain[ft].values-fMean[ft])/(fStd[ft])
#                     regionalPredictSet[ft]=(regionalPredictSet[ft].values-fMean[ft])/(fStd[ft])


      
            le=LabelEncoder()
            regionalTrain['le']=le.fit_transform(regionalTrain['place_id'].values)

#             s_w=1**(15-regionalTrain['day'].values/30.0)
            
            trainingStartTime=time.time()
            if model=='rf':
#                 s_w_rf=3**(-regionalTrain['month'].values*30)
#                 s_w_rf=1.65-regionalTrain['day'].values
                s_w_rf=np.ones(len(regionalTrain))

    #             clf=RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features=None).fit(regionalTrain[f], regionalTrain['le'])   
            
                clf=RandomForestClassifier(n_jobs=-1, n_estimators=150, random_state=0).fit(regionalTrain[f], regionalTrain['le'], sample_weight=s_w_rf)  

            if model=='xgb':
                clf=XGBClassifier(learning_rate=0.02, n_estimators=220, objective='multi:softprob', max_depth=3, seed=0).fit(regionalTrain[f], regionalTrain['le'])
            
            
            if model=='knn':
                def calculate_distance(distances):
                    return distances ** -2
#                 clf=KNeighborsClassifier(n_neighbors=25, weights='distance',metric='manhattan', n_jobs=-1).fit(regionalTrain[f], regionalTrain['le'])
                numNeighbors=np.floor(np.sqrt(len(regionalPredictSet))/5.1282).astype(int)
#     np.floor(0.3*np.sqrt(len(regionalPredictSet)))
#                 numNeighbors=36


                clf=KNeighborsClassifier(n_neighbors=numNeighbors, weights=calculate_distance, metric='manhattan', n_jobs=-1).fit(regionalTrain[f], regionalTrain['le'])

         
            trainingTimes.append(time.time()-trainingStartTime)
            
            
            if modelSerialization:
                modelFileName='{:04d}-{}-{}-{}-{}.clf'.format(count, xStart, xEnd, yStart, yEnd)
                modelFilePath=folderPath+modelFileName
                with open(modelFilePath, 'ab+') as fo:
                    pickle.dump(clf, fo, pickle.HIGHEST_PROTOCOL)
            
            
            predictionStartTime=time.time()
            prob=clf.predict_proba(regionalPredictSet[f])
            predictionTimes.append(time.time()-predictionStartTime)
            
            pred=[sorted(zip(p, clf.classes_))[-3:][::-1] for p in prob]
            prediction=le.inverse_transform([zip(*p)[1] for p in pred])
            confidence=[zip(*p)[0] for p in pred]
            
            
                
            fbscoreForSerialization=-1
            if 'place_id' in predictSet.columns:
                regionalAccuracy=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                regionalConfidence=[sum(z)/len(regionalPredictSet) for z in zip(*confidence)]
                regionalConfidence3=sum(regionalConfidence)

                fbAccuracy=0
                fbAccuracy+=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][:1] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/2*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][1:2] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/3*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][2:3] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                FBscores.append(fbAccuracy)
                print 'region {}: {},{} accuracy: {},  fbAccu: {}, confidence: {}:'.format(count, xNum, yNum, regionalAccuracy, fbAccuracy, regionalConfidence3)
                accuracies.append(regionalAccuracy)
                if resultSerialization:
                    fbscoreForSerialization=fbAccuracy
                
            if resultSerialization:
                resultFileName='{:04d}-{}-{}-{}-{}.rst'.format(count, xStart, xEnd, yStart, yEnd)
                resultFilePath=folderPath+resultFileName
                predColumns=zip(*prediction)
                confColumns=zip(*confidence)

                
                if len(predColumns[2])!=len(prediction):
                    print 'missing values',count, len(predColumns[0])
                
                results=pd.DataFrame({'originalIndex': regionalPredictSet['originalIndex'].tolist(),\
                                      'x':regionalPredictSet['x'].tolist(), 'y':regionalPredictSet['y'].tolist(), \
                                      'accuracy':regionalPredictSet['accuracy'].tolist(), 'pred0':predColumns[0], \
                                      'pred1':predColumns[1], 'pred2':predColumns[2], 'conf0': confColumns[0], \
                                      'conf1': confColumns[1], 'conf2': confColumns[2], 'regionalFBScore': [fbscoreForSerialization]*len(regionalPredictSet)})

                results.to_csv(resultFilePath)



        count+=1
        if count%10==0:

            print '{} : total time {} s.'.format(count, time.time()-startTime)
            print 'Average training Time: ', sum(trainingTimes)/len(trainingTimes)
            print 'Average prediction Time: ', sum(predictionTimes)/len(predictionTimes)
            print 'Average FB Score', np.mean(FBscores)
            print 
            
            startTime=time.time()
            trainingTimes=[]
            predictionTimes=[]


    print 
    print np.mean(accuracies)
    print np.var(accuracies)
    print
    print np.mean(FBscores)
    print testNum
    print 'done'
    
    
    
    

In [8]:
train_test(modelSerialization=False, model='rf', resultSerialization=True, predictSet=test, th=8, xMargin=0.025, yMargin=0.0125)

10 : total time 15.0767810345 s.
Average training Time:  0.0234433174133
Average prediction Time:  0.355998563766
Average FB Score nan



/home/zhenjie/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


20 : total time 13.5254030228 s.
Average training Time:  0.0220759153366
Average prediction Time:  0.361755752563
Average FB Score nan

30 : total time 12.3426029682 s.
Average training Time:  0.0206168174744
Average prediction Time:  0.321492004395
Average FB Score nan

40 : total time 12.4514620304 s.
Average training Time:  0.0212047815323
Average prediction Time:  0.293279361725
Average FB Score nan

50 : total time 16.0946509838 s.
Average training Time:  0.024032831192
Average prediction Time:  0.420702910423
Average FB Score nan

60 : total time 15.4576790333 s.
Average training Time:  0.0238263845444
Average prediction Time:  0.348551893234
Average FB Score nan

70 : total time 15.6308779716 s.
Average training Time:  0.0231595516205
Average prediction Time:  0.383421730995
Average FB Score nan

80 : total time 14.8638629913 s.
Average training Time:  0.0226248979568
Average prediction Time:  0.351394581795
Average FB Score nan

90 : total time 15.5369110107 s.
Average training

/home/zhenjie/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [9]:
def makeSubmission(resultFolder):
    results=[]
    for f in os.listdir(resultFolder):
        if f.endswith('.rst'):
            fi=resultFolder+'/'+f
            results.append(pd.read_csv(fi))
    results=pd.concat(results)  
        
    results.sort(['originalIndex'], inplace=True)
    return results

In [10]:
results=makeSubmission('../submissions/knn-02/')
predictions=results[['pred0','pred1','pred2']]
submit=pd.DataFrame()
# submit.loc[:,'row_id']=np.arange(len(predictions))
submit.loc[:,'place_id']=predictions[['pred0', 'pred1', 'pred2']].apply(lambda x: ' '.join([str(nx) for nx in x]), axis=1)
submit.loc[:,'row_id']=np.arange(len(predictions))


/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [11]:
submit=submit[['row_id','place_id' ]]
submit.set_index('row_id', inplace=True)
submit.to_csv('../submissions/knn-02.csv')
